<a href="https://colab.research.google.com/github/Giuseppe86-lab/Software-Gestione-Magazzino/blob/main/Soft_Gest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

class Wms():
  """
  This Class defines the operations that can be done by the software
  warehouse management.
  """
  def __init__(self, inventory, _gross = 0, _net = 0):
      """
      Initialize the Wms instance.

      Methods:
      - select_new_operation(self, AVEILABLE_COMMAND)
      - add_product(self)
      - list_products(self)
      - sell_product(self)
      - profits(self)
      - delete_product(self, product)
      - check_pos_value(self, product)
      - help(self)
      - close(self)


      Arguments:
      - inventory (dict): dictionary that contains products as keys and
      the quantity, purchase price and sales price as values.
      - _gross (float): sales revenue of the products in the inventory, value
      default zero.
      - _net (float): revenue from sales of warehouse products,
      default value zero.
      """

      self.inventory = inventory
      self._gross = _gross
      self._net = _net

  def select_new_operation(self, AVEILABLE_COMMAND):
    """
    Asks the user to select an operation and checks that it is among those
    defined in the methods of the class. If the input entered is not listed.
    prints an error message.

    Arguments:
    - AVEILABLE_COMMAND (list): list with all user-executable commands.
    """
    try:
      command = input("\nInserisci un comando: ")
      assert(command in AVEILABLE_COMMAND), 'Comando non valido'
      return command
    except AssertionError as e:
      print(e)
      self.help()

  def add_product(self):
    """
    It asks the user if he/she wants to add a product, if the product is not in
    the warehouse, will be asked in addition to the quantity the price of purchase
    and sale. If the product is already in the warehouse, only the quantity.

    The function checks that the quantities and prices entered by the user are
    numbers.

    The function calls the self.check_pos_value(self, value) method to check that the
    quantities and prices are positive numbers.
    """
    product = input('Nome del prodotto: ')
    if product.lower() in self.inventory.keys():
      try:
        amount = int(input('Quantità: '))
        if self.check_pos_value(amount):
          self.inventory[product.lower()]['Quantità'] += amount
          print(f"AGGIUNTO: {amount} X {product.lower()}")
        else:
          print("la quantità dev'essere un numero intero positivo")

      except ValueError:
        print("Impossibile aggiungere il prodotto: la quantità dev'essere"
              " un numero intero")

    else:
      try:
        amount = int(input('Quantità: '))
        cost = float(input('Prezzo di acquisto: '))
        price = float(input('Prezzo di vendita: '))

        if self.check_pos_value(amount) and self.check_pos_value(cost) and self.check_pos_value(price):
          self.inventory[product.lower()] = {}
          self.inventory[product.lower()]['Quantità'] = amount
          self.inventory[product.lower()]['Prezzo di acquisto'] = cost
          self.inventory[product.lower()]['Prezzo di vendita'] = price
          print(f"AGGIUNTO: {self.inventory[product.lower()]['Quantità']} X {product.lower()}")
        else:
          print("La quantità, il prezzo di acquisto e il prezzo di vendita devono"
                " essere numeri positivi")
      except ValueError:
        print("Impossibile aggiungere il prodotto: La quantità dev'essere"
              " un numero intero positivo, mentre i prezzi dei numeri positivi")


  def list_products(self):
    """
    Print a table with the list of products in stock, the quantity and the
    selling price.
    If there are no products in stock, it prints an error message.
    """
    try:
      max_length_product_name = max(len(product) for product in self.inventory.keys())
      print(f"{'PRODOTTO':<{max_length_product_name+5}}\tQUANTITA'\tPREZZO")
      for product_name, product_info in self.inventory.items():
        print(f"{product_name:<{max_length_product_name+5}}\t{product_info['Quantità']:<10}\t€{product_info['Prezzo di vendita']:<10}")
    except ValueError:
      print('Il magazzino è vuoto')

  def sell_product(self):
    """
    It asks the user which product he/she wants to buy and the quantity, checks
    whether the goods are in stock and whether the quantity required is available.

    Allows the user to buy several products one after another and at the end
    prints a report of the goods purchased and the price paid.

    At the same time it updates stock availability. If the goods are
    finished it calls the delete_products(self, product) method to delete it.
    """
    selling_on = 'si'
    record_selling = {}
    total_revenue = 0
    total_cost = 0
    while selling_on == 'si':

      product = input('Nome del prodotto: ')
      try:

        assert(product in self.inventory.keys()), 'Prodotto non presente in magazzino'
        n_product = int(input('Quantità: '))
        assert(n_product <= self.inventory[product]['Quantità']), f"Max disponibilità {self.inventory[product]['Quantità']}"
        record_selling[product]= [n_product, self.inventory[product]['Prezzo di vendita']]
        total_revenue += n_product*self.inventory[product]['Prezzo di vendita']
        total_cost += n_product*self.inventory[product]['Prezzo di acquisto']
        self.inventory[product]['Quantità'] -= n_product

        if self.inventory[product]['Quantità'] == 0:
          self.delete_product(product)

        selling_on = input('Aggiungere un altro prodotto ? (si/no):')

      except AssertionError as e:
        print(e)

      except ValueError:
        print("La quantità dev'essere un numero intero")

    self._gross += total_revenue
    self._net += total_revenue - total_cost
    print("VENDITA REGISTRATA")
    for product_name, product_info in record_selling.items():
      print(f'- {product_info[0]} X {product_name}: €{product_info[1]}')
    print(f'Totale: €{total_revenue:.2f}')

  def profits(self):
    """
    Print both gross and net profits achieved during the current session
    of using the management software.
    """
    print(f"Profitto: lordo=€{self._gross:.2f} netto=€{self._net:.2f}")

  def delete_product(self, product):
    """
    Delete products from the warehouse dictionary, delete all information
    related to the product.

    Arguments:
    - product(str): name of the product you want to delete.
    """
    del self.inventory[product]

  def check_pos_value(self, value):
    """
    Checks that the values entered by the user are positive.

    Arguments:
    - value(float/int): value you want to check.

    Returns True if value is positive, otherwise False
    """
    if value > 0:
      return True
    else:
      return False

  def help(self):
    """
    Prints a list of all available commands, giving a brief description of them.
    """
    return print("I comandi disponibili sono i seguenti:\n"
    "aggiungi: aggiungi un prodotto al magazzino\n"
    "elenca: elenca i prodotti in magazzino\n"
    "vendita: registra una vendita effettuata\n"
    "profitti: mostra i profitti totali\n"
    "aiuto: mostra i possibili comandi\n"
    "chiudi: esci dal programma\n")

  def close(self):
    """
    Saves the dictionary with the inventory in a .json file for later sessions.
    Prints a greeting message.
    """
    with open("inventory_veg_shop.json", 'w', encoding="utf-8") as json_file:
      json.dump(self.inventory, json_file, ensure_ascii=False, indent = 6)
    print("Bye bye\n")


def load_inventory():
  """
  Search the working directory to see if the file with data from a previous
  session is present. If it is present, it reads the .json file, and returns
  the dictionary.
  If it is not present it initializes a dictionary and returns it.
  """

  try:
    with open("inventory_veg_shop.json", encoding="utf-8") as json_file:
      inventory = json.load(json_file)

  except FileNotFoundError:
    inventory = {}

  return inventory

"""
This part represents the main of the management software; it defines:
- the list of accessible commands;
- calls the function to load the inventory;
- initializes the Wms object;
- asks the user which command he wants to execute and, if the command is valid, calls
the corresponding methods.
"""

AVEILABLE_COMMAND = ['aggiungi', 'elenca', 'vendita', 'profitti', 'aiuto', 'chiudi']


opened_inventory = load_inventory()
my_inventory = Wms(opened_inventory)

closure = False

while not closure:

  command = my_inventory.select_new_operation(AVEILABLE_COMMAND)

  if command == 'aiuto':
    my_inventory.help()

  if command == 'aggiungi':
    my_inventory.add_product()

  if command == 'elenca':
    my_inventory.list_products()

  if command == 'vendita':
    my_inventory.sell_product()

  if command == 'profitti':
    my_inventory.profits()

  if command == 'chiudi':
    my_inventory.close()
    closure = True




Inserisci un comando: elenca
PRODOTTO          	QUANTITA'	PREZZO
latte di soia     	15        	€1.4       
tofu              	8         	€4.19      

Inserisci un comando: chiudi
Bye bye

